In [ ]:
%pip install -r requirements.txt

### Überblick
In diesem Projekt simulieren mehrere KI-Agenäten einen lebendigen Buchclub. Sie diskutieren, welches Buch als nächstes gemeinsam gelesen werden soll. Jeder Agent bringt eigene Perspektiven ein, reagiert auf die Argumente der anderen und beteiligt sich aktiv an der Entscheidungsfindung. Ziel ist es, einen diskursiven, dynamischen Gesprächsverlauf zu ermöglichen – ganz wie unter echten Menschen